In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

from tables import *

# A heading for Voila to render 

## A subheading for Voila to render

- Voila! It works ... for now. 
- Only change the bottom two sliders to positive values as using a reduced data set.
- The responsiveness doesn't seem to be to bad. This is using base matplotlib to plot.

In [ ]:
class App:
    
    def __init__(self, h5file_data):
        
        # Initialise data and plot widget. 
        self._h5file_data = h5file_data
        self._kpoints = h5file_data.get_node('/VL2_data', 'k_points')
        self._plot_container = widgets.Output()

        # Define the variable parameter sliders. 
        self._potential_height_slider, potential_height_slider_box = self._create_parameter_slider(
            min_val=0.0, max_val=0.5, step_val=0.10, default_val=0.0, model_param=r'$V_H$')
        self._tp_slider, tp_slider_box = self._create_parameter_slider(
            min_val=-0.5, max_val=0.5, step_val=0.05, default_val=-0.05, model_param=r"$t'$")
        self._tsoc_slider, tsoc_slider_box = self._create_parameter_slider(
            min_val=-0.2, max_val=0.2, step_val=0.02, default_val=0.0, model_param=r"$t_{soc}$")
        self._deltaU1_slider, deltaU1_slider_box = self._create_parameter_slider(
            min_val=-0.5, max_val=0.5, step_val=0.1, default_val=0.0, model_param=r"$\Delta U_1$")
        self._deltaU2_slider, deltaU2_slider_box = self._create_parameter_slider(
            min_val=-0.5, max_val=0.5, step_val=0.1, default_val=0.0, model_param=r"$\Delta U_2$")
        
        # Define the arrangement of the Output (plot_container) and slider widgets.
        self.container = widgets.VBox([
            self._plot_container,
            potential_height_slider_box, 
            tp_slider_box,
            tsoc_slider_box,
            deltaU1_slider_box,
            deltaU2_slider_box
        ], layout=widgets.Layout(align_items='center'))
        
        # Initialise the plot with default values for sliders. 
        self._update_app()

    
    @classmethod
    def from_file(cls, filename):
        data = open_file(filename, 'r')
        return cls(data) 
    
    def _create_indicator_dropdown(self, indicators, initial_index):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        return dropdown
    
    def _create_parameter_slider(self, min_val, max_val, step_val, default_val, model_param):
        parameter_slider = widgets.FloatSlider(
            min=min_val, max=max_val, step=step_val, value=default_val, 
            description=model_param+': ', layout=widgets.Layout(width='500px')
        )
        parameter_slider.observe(self._on_change, names=['value'])
        parameter_slider_box = widgets.HBox([parameter_slider])
        return parameter_slider, parameter_slider_box
    
    def _get_bands(self, potential_height, tp, tsoc, deltaU1, deltaU2):
        t = -1.0  # fix nearest-neighbour hopping parameter 
        potential_period = 2.0  # potential_period fixed for now
        params_path_potential = 'VL' + '{0:.0f}'.format(potential_period) + '_VH' + '{0:.3f}'.format(potential_height)
        params_path_hoppings = '_t' + '{0:.2f}'.format(t) + '_tp' + '{0:.2f}'.format(tp) + \
                               '_tsoc' + '{0:.2f}'.format(tsoc)
        params_path_onsite = '_deltaU1' + '{0:.2f}'.format(deltaU1) + '_deltaU2' + '{0:.2f}'.format(deltaU2)
        bands_filename = params_path_potential + params_path_hoppings + params_path_onsite
        bands = self._h5file_data.get_node('/VL2_data', 'bands_'+bands_filename)
        return bands
    
    def _create_plot(self, potential_height_value, tp_value, tsoc_value, deltaU1_value, deltaU2_value):
        bands = self._get_bands(potential_height_value, tp_value, tsoc_value, 
                                deltaU1_value, deltaU2_value)
        plt.figure()
        for band_ind in range(len(bands[0])):
            plt.plot(self._kpoints[:], bands[:, band_ind], color=default_blue)
        plt.xlabel(r'$k$ path')
        plt.ylabel(r'$E$ (eV)')
        
    def _on_change(self, _):
        self._update_app()
        
    def _update_app(self):
        potential_height_value = self._potential_height_slider.value
        tp_value = self._tp_slider.value
        tsoc_value = self._tsoc_slider.value
        deltaU1_value = self._deltaU1_slider.value
        deltaU2_value = self._deltaU2_slider.value
        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            self._create_plot(potential_height_value, tp_value, tsoc_value, deltaU1_value, deltaU2_value)
            plt.show()

In [ ]:
default_blue = '#1f77b4'  # default matplotlib blue
app = App.from_file('all_data.h5')

app.container